In [1]:
import torch
import torch.nn as nn
import pandas as pd
import os
import numpy as np
from torch.utils.data import DataLoader

In [2]:
# read the datasets. They are divided in test and train
train = pd.read_csv('../data/train.csv')
print(f'Lenght of train data is {len(train):,} observations')
# drop missing values
train = train.dropna()
print(f'Lenght of train data is {len(train):,} observations')
train.head(7)

Lenght of train data is 27,481 observations
Lenght of train data is 27,480 observations


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive


In [3]:
train = train.iloc[0:1000]
# split dataset into training and validation 
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(train,
                              random_state = 40,
                              stratify = train[['sentiment']],
                              test_size = 0.1)

In [4]:
# find max text span
#test['text'].str.len().max()

In [5]:
# repeat for test
test = pd.read_csv('../data/test.csv')
print(f'Lenght of the test data is {len(test):,} observations')

Lenght of the test data is 3,534 observations


In [6]:
#train_data = train_data.iloc[44:48]
#train_data

In [7]:
# test
from transformers import BertTokenizer
max_seq_length = 180
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


class Tweet_Loader(object):
    '''
    1. tokenize the text
    2. find the indices of the selected text in the original text
    3. append the question to the original text and modify original indexes
    4. identify start index and end index
    5. identify start and end index
    '''

    def __init__(self, text, selected_text, sentiment, textID):
        
        # define the text column from the dataframe
        self.text_column = text.tolist()
    
        # define sentiment
        self.sentinent = sentiment.tolist()
        
        # define selected text
        self.selected_text = selected_text.tolist()
    
        # define the label column and transform it to list
        self.textID = textID.tolist()
        
    # iter method to get each element at the time and tokenize it using bert        
    def __getitem__(self, index):
        
        sentinent = self.sentinent[index]
        text_id = self.textID[index]
        text = self.text_column[index]
        selected_text = self.selected_text[index]
        # get tokenized text
        tokenized_text = tokenizer.tokenize(text)
        tokenized_selected_text = tokenizer.tokenize(selected_text)
        
        # find the spans of the answer
        start_span_index = -1
        end_span_index = -1
        
        len_sel_text = len(tokenized_selected_text)
        
        # define the start and end index of the selected text. We iterate over the original tweet and
        # identify the point where the first word of the original text matches the first word of the selected
        # text. Then if traversing the substring of original tweet is the same as the selected tweet
        # then get the start and end spans
        for index in (i for i, e in enumerate(tokenized_text) if e == tokenized_selected_text[0]):
            if tokenized_text[index: index+len_sel_text] == tokenized_selected_text:
                start_span_index = index
                end_span_index = index + len_sel_text - 1
                break
        
        # need to define a function to handle cases where there are strings of lenght greater than 
        # the max allowed.
        
        # use the encode plus function
        dictionary_inputs = tokenizer.encode_plus(self.sentinent[index],
                                                  tokenized_text, max_length = max_seq_length,
                                                  truncation_strategy  ='do_not_truncate', 
                                                  pad_to_max_length = True, padding_side='Right', 
                                                  is_pretokenized = True, return_token_type_ids=True,
                                                  return_attention_mask=True,
                                                  return_tensors = 'pt')
        
        # create the input by adding the CLS token, the SEP token for separaring the question and the end of the
        # whole sentence
        #input_sentence = ['CLS'] + [sentinent] + ['SEP'] + tokenized_text + ['SEP']
        start_span_index += 3
        end_span_index +=  3
        
        # convert to ids
        #input_ids = tokenizer.convert_tokens_to_ids(input_sentence)

        # create the padding
        #padding = [0] * (max_seq_length - len(input_sentence))
        
        # add padding if needed
        #input_ids += padding
        
        # create attention masks
        #attention_masks = [1 if x>0 else 0 for x in input_ids] 
        
        # do we need token type?
        # add token type
        
        return [dictionary_inputs, start_span_index, end_span_index, text_id]
  
    def __len__(self):
        return len(self.text_column)

In [8]:
#train_data


In [9]:
# create a class to process the traininga and test data
batch_size = 4
training_data = Tweet_Loader(train_data['text'],
                             train_data['selected_text'], 
                             train_data['sentiment'],
                             train_data['textID'])

valid_data =  Tweet_Loader(val_data['text'],val_data['selected_text'], val_data['sentiment'], val_data['textID'])

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(training_data, batch_size=batch_size, shuffle=False, num_workers=4),
                    'val': DataLoader(valid_data, batch_size=batch_size, shuffle=True, num_workers=4)
                   }

dataset_sizes = {'train':len(training_data),
                 'val':len(valid_data)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(device)

In [10]:
'''
from torch.utils.data import Dataset, DataLoader

a = Tweet_Loader(train['text'],train['selected_text'], train['sentiment'], train['textID'])

# use the dataloaders class to load the data
dataloaders_dict = {'train': DataLoader(a, batch_size=1, shuffle=True, num_workers=4)}

dataset_sizes = {'train':len(a)}
'''

"\nfrom torch.utils.data import Dataset, DataLoader\n\na = Tweet_Loader(train['text'],train['selected_text'], train['sentiment'], train['textID'])\n\n# use the dataloaders class to load the data\ndataloaders_dict = {'train': DataLoader(a, batch_size=1, shuffle=True, num_workers=4)}\n\ndataset_sizes = {'train':len(a)}\n"

In [11]:
#train.loc[44]['text']
example = next(iter(dataloaders_dict.get('train')))


print(example[1],example[2])

tensor([11,  3,  2,  3]) tensor([12,  5,  2, 13])


In [12]:
example[0].get('input_ids')[0][0]

tensor([  101,  3893,   102,  1045,  6655,  2049,  4658,  2091,  1999,  5034,
         1010,  9616,  1029,  2009,  1036,  1055,  2025,  2182,  1012,  1045,
         4033,  1036,  1056,  3030, 18972,  2144, 11501,  2043,  3611,  1998,
         1045,  2020, 14743,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [13]:
def reconstruct_string(start_index, end_index, input_ids):
    # get the actual span tokes using the start_span and end_span
    original_tokens = input_ids[start_index:end_index+1]
    original_tokens = tokenizer.convert_ids_to_tokens(original_tokens)
    # move to string 
    original_string = ' '.join([x for x in original_tokens])

    return original_string
    
reconstruct_string(example[1][0].item(), example[2][0].item(), example[0].get('input_ids')[0][0])
    

'huh ?'

In [14]:
#tokens_ =  tokenizer.convert_ids_to_tokens(example[0].get('input_ids')[0][0][example[0][0]:example[2][0]])
#tokens_
#example[0].get('input_ids')[0][0]
#a= example[1][0].item()
#b= example[2][0].item()+1
#print(example[0].get('input_ids')[0][0][example[1][0].item():example[2][0].item()+1]
#     )
#text_r = ' '.join([x for x in tokens_])
#print(text_r)
#fine_text = text.replace(' ##', '')

In [15]:
#a = tokenizer.tokenize('sigh.. stayed up late cause was on a role w/ assessment, now I`m so tired my eyes r stinging & my head hurts.. & I still need 500 words')
#len(a)
# import the model
from transformers import BertForQuestionAnswering, BertConfig
config  = BertConfig()
model = BertForQuestionAnswering.from_pretrained(
    "bert-base-uncased", config = config)

In [16]:
# check the model
model

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [17]:
# define jaccard metric
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [18]:
'''
# ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')
    
'''

'\n# ======== Reconstruct Answer ========\n    # Find the tokens with the highest `start` and `end` scores.\n    answer_start = torch.argmax(start_scores)\n    answer_end = torch.argmax(end_scores)\n\n    # Get the string versions of the input tokens.\n    tokens = tokenizer.convert_ids_to_tokens(input_ids)\n\n    # Start with the first token.\n    answer = tokens[answer_start]\n\n    # Select the remaining answer tokens and join them with whitespace.\n    for i in range(answer_start + 1, answer_end + 1):\n        \n        # If it\'s a subword token, then recombine it with the previous token.\n        if tokens[i][0:2] == \'##\':\n            answer += tokens[i][2:]\n        \n        # Otherwise, add a space then the token.\n        else:\n            answer += \' \' + tokens[i]\n\n    print(\'Answer: "\' + answer + \'"\')\n    \n'

In [19]:
import logging
logging.basicConfig(level = logging.INFO, filename ='../logs/bert_tweet_extraction_160_nofinetune.txt', filemode ='w', 
                   format = '%(name)s -%(levelname)s - %(message)s')

In [20]:
import torch.optim as optim 
from torch.optim import lr_scheduler, AdamW
#criterion = nn.CrossEntropyLoss()


#lrmain = 1e-3

optim1 = optim.AdamW(model.parameters(), lr=3e-5, betas=(0.9, 0.999))
    #[
    #    {"params":model.bert.parameters(),"lr": lrmain},
   #])

optimizer_ft = optim1

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.1)

In [21]:
import torch.nn.functional as F
#import torch.nn.functional as F
import copy 
import time
number_steps = 10
print(number_steps)

def train_model(model, optimizer, scheduler, num_epochs=3):
    best_eval_jac = 0.0
    for epoch in range(num_epochs):
        #zero the model gradients
        model.zero_grad()
        print(f'starting epoch {epoch+1} out of {num_epochs}')
        training_loss = []
        training_jaccard = []
        val_loss = []
        val_jaccard = []
        outputs_ = []
        labels_ = []

        epoch_loss = 0.0

        # track number of batches and number of iterations
        counter = 0
        iterations = 0

        # Iterate over data, feeding inputs, attention masks and labels
        model.train()
        for i, (dict_param, start_position, end_position, tweet_id) in enumerate(dataloaders_dict['train']):
            # add a counter that will register how many examples we have fed to the
            # model
            counter+= batch_size
            iterations+=1
            # move the sequences, labels and masks to the GPU
            inputs = dict_param.get('input_ids').to(device).squeeze()
            #print(inputs.shape)
            token_type_ids = dict_param.get('token_type_ids').to(device).squeeze()
            #print(token_type_ids.shape)
            attention_mask = dict_param.get('attention_mask').to(device).squeeze()
            #print(attention_mask.shape)
            start_pos = start_position.to(device)
            end_pos = end_position.to(device)
            

            # feed the sequences to the model, specifying the attention mask
            loss, start_scores, end_scores = model(inputs, token_type_ids = token_type_ids, 
                                                   attention_mask = attention_mask, 
                                                   start_positions = start_pos,
                                                   end_positions = end_pos)
            
            # 
            training_loss.append(loss.item())
            epoch_loss+=loss.item()
            loss.backward()
            #print(start_scores, start_scores.shape)
            
                        
            # update every number of accumulation
            if (i+1) % number_steps == 0:
                
                optimizer.step()                            # Now we can do an optimizer step
                model.zero_grad()                           # Reset gradients tensors
      
            # only present the information 
            if counter%100 == 0:
                
                start_preds=torch.argmax(start_scores,dim=1).cpu().detach().numpy()
                end_preds=torch.argmax(end_scores,dim=1).cpu().detach().numpy()
                start_index = start_pos.cpu().cpu().detach().numpy()
                end_index = end_pos.cpu().cpu().detach().numpy()
                inputs = inputs.cpu().detach().numpy()
                       
                for i in range(inputs.shape[0]):
                    original_string = reconstruct_string(start_index[i], end_index[i], inputs[i])
                    pred_string = reconstruct_string(start_preds[i], end_preds[i], inputs[i])
                    jaccard_score = jaccard(original_string,pred_string)
                    training_jaccard.append(jaccard_score)
                    #print(f'the original is {original_string}, the predicted is {pred_string}')
                    
                
                print(f'current loss is {epoch_loss/iterations} and jaccard is {np.mean(training_jaccard):,.2%}')
                
        with torch.no_grad():
            model.eval()
            counter_val = 0
            iterations_val = 0
            
            for i, (dict_param, start_position, end_position, tweet_id) in enumerate(dataloaders_dict['train']):
            # add a counter that will register how many examples we have fed to the
            # model
                counter+= batch_size
                iterations_val+=1
            # move the sequences, labels and masks to the GPU
                inputs = dict_param.get('input_ids').to(device).squeeze()
                token_type_ids = dict_param.get('token_type_ids').to(device).squeeze()
                attention_mask = dict_param.get('attention_mask').to(device).squeeze()
                start_pos = start_position.to(device)
                end_pos = end_position.to(device)

            # feed the sequences to the model, specifying the attention mask
                loss, start_scores, end_scores = model(inputs, token_type_ids = token_type_ids, 
                                                   attention_mask = attention_mask, 
                                                   start_positions = start_pos,
                                                   end_positions = end_pos)
            
                val_loss.append(loss.item())
                
                if counter%10 == 0:

                    start_preds=torch.argmax(start_scores,dim=1).cpu().detach().numpy()
                    end_preds=torch.argmax(end_scores,dim=1).cpu().detach().numpy()
                    start_index = start_pos.cpu().cpu().detach().numpy()
                    end_index = end_pos.cpu().cpu().detach().numpy()
                    inputs = inputs.cpu().detach().numpy()
                       
                    for i in range(inputs.shape[0]):
                        original_string = reconstruct_string(start_index[i], end_index[i], inputs[i])
                        pred_string = reconstruct_string(start_preds[i], end_preds[i], inputs[i])
                        jaccard_score = jaccard(original_string,pred_string)
                        val_jaccard.append(jaccard_score)
                        
                
                        print(f'current validation loss is {np.sum(val_loss)/iterations_val} and'
                              f'jaccard score is {np.mean(val_jaccard):,.2%}')
                              
        print(f'For epoch {epoch+1} training loss is {np.sum(training_loss)/iterations}, \
        training accuracy is {np.mean(training_jaccard):,.2%}, Validation \
        loss is {np.sum(val_loss)/iterations_val} and validation accuracy is {np.mean(val_jaccard):,.2%}')
        jac_acc = np.mean(val_jaccard)
        if jac_acc >= best_eval_jac:
            best_eval_jac = jac_acc
            print(f'saving the model with jac_acc  {jac_acc:,.2%} ')
            torch.save(model.state_dict(), '../models_trained/bert_tweet_extraction_no_finetuned.pth')
            model.config.to_json_file('../models_trained/bert_base_uncased.json')
            torch.save(model, '../models_trained/bert_tweet_extraction_no_finetuned.bin')
            #torch.save(optimizer_ft.state_dict(), 
            #           'bert_tweet_classification_state_dict_280_no_finetuned_optimizer.pth')
            #model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
            #output_model_file = os.path.join("bert_tweet_classification_280_no_finetuned_model.bin")

        else:
            print(f'model did not improve')
        
        logging.info(f'We completed epoch {epoch+1} with a training loss of {np.sum(training_loss)/iterations} \
        a jac score of {np.mean(training_jaccard):,.2%}, Validation \
        loss is {np.sum(val_loss)/iterations_val} and validation accuracy is {np.mean(val_jaccard):,.2%}')
      
    return model

10


In [22]:
# time to train 
model.to(device)
model_ft1 = train_model(model, optimizer_ft, exp_lr_scheduler,
                       num_epochs=3)

starting epoch 1 out of 3
current loss is 5.121322021484375 and jaccard is 10.62%
current loss is 4.932855062484741 and jaccard is 25.06%
current loss is 4.711866111755371 and jaccard is 28.86%
current loss is 4.434444897174835 and jaccard is 41.28%
current loss is 4.163025953292847 and jaccard is 37.93%
current loss is 3.935214451154073 and jaccard is 42.51%
current loss is 3.7238373960767475 and jaccard is 39.38%
current loss is 3.5585721600055695 and jaccard is 39.30%
current loss is 3.426396670871311 and jaccard is 37.93%
current validation loss is 2.283890438079834 andjaccard score is 0.00%
current validation loss is 2.283890438079834 andjaccard score is 1.61%
current validation loss is 2.283890438079834 andjaccard score is 27.74%
current validation loss is 2.283890438079834 andjaccard score is 45.81%
current validation loss is 2.251836657524109 andjaccard score is 56.65%
current validation loss is 2.251836657524109 andjaccard score is 47.20%
current validation loss is 2.251836657

current validation loss is 2.124443975810347 andjaccard score is 45.85%
current validation loss is 2.124443975810347 andjaccard score is 45.44%
current validation loss is 2.124443975810347 andjaccard score is 45.66%
current validation loss is 2.124443975810347 andjaccard score is 46.13%
current validation loss is 2.122585780620575 andjaccard score is 46.59%
current validation loss is 2.122585780620575 andjaccard score is 47.04%
current validation loss is 2.122585780620575 andjaccard score is 46.84%
current validation loss is 2.122585780620575 andjaccard score is 47.28%
current validation loss is 2.115283866082468 andjaccard score is 47.09%
current validation loss is 2.115283866082468 andjaccard score is 46.81%
current validation loss is 2.115283866082468 andjaccard score is 47.24%
current validation loss is 2.115283866082468 andjaccard score is 47.67%
current validation loss is 2.114247943460941 andjaccard score is 47.29%
current validation loss is 2.114247943460941 andjaccard score is

current validation loss is 1.4893635771491311 andjaccard score is 49.70%
current validation loss is 1.4893635771491311 andjaccard score is 50.90%
current validation loss is 1.4893635771491311 andjaccard score is 52.04%
current validation loss is 1.4893635771491311 andjaccard score is 53.13%
current validation loss is 1.4866054823001227 andjaccard score is 52.20%
current validation loss is 1.4866054823001227 andjaccard score is 53.24%
current validation loss is 1.4866054823001227 andjaccard score is 54.23%
current validation loss is 1.4866054823001227 andjaccard score is 55.18%
current validation loss is 1.4868202484571016 andjaccard score is 54.29%
current validation loss is 1.4868202484571016 andjaccard score is 53.33%
current validation loss is 1.4868202484571016 andjaccard score is 53.71%
current validation loss is 1.4868202484571016 andjaccard score is 54.60%
current validation loss is 1.4552634971482412 andjaccard score is 55.23%
current validation loss is 1.4552634971482412 andja

current validation loss is 1.4156361425795205 andjaccard score is 53.83%
current validation loss is 1.4156361425795205 andjaccard score is 54.12%
current validation loss is 1.4156361425795205 andjaccard score is 54.08%
current validation loss is 1.4156361425795205 andjaccard score is 53.75%
current validation loss is 1.4163046885104407 andjaccard score is 54.03%
current validation loss is 1.4163046885104407 andjaccard score is 54.30%
current validation loss is 1.4163046885104407 andjaccard score is 54.58%
current validation loss is 1.4163046885104407 andjaccard score is 54.25%
current validation loss is 1.422064029061517 andjaccard score is 54.08%
current validation loss is 1.422064029061517 andjaccard score is 54.35%
current validation loss is 1.422064029061517 andjaccard score is 54.62%
current validation loss is 1.422064029061517 andjaccard score is 54.88%
current validation loss is 1.4300026023929768 andjaccard score is 54.88%
current validation loss is 1.4300026023929768 andjaccar

current validation loss is 0.9864160672478054 andjaccard score is 62.55%
current validation loss is 0.9864160672478054 andjaccard score is 61.93%
current validation loss is 0.9864160672478054 andjaccard score is 62.35%
current validation loss is 0.9864160672478054 andjaccard score is 62.76%
current validation loss is 0.9842836583654085 andjaccard score is 63.16%
current validation loss is 0.9842836583654085 andjaccard score is 62.80%
current validation loss is 0.9842836583654085 andjaccard score is 62.33%
current validation loss is 0.9842836583654085 andjaccard score is 61.77%
current validation loss is 0.976834801197052 andjaccard score is 62.17%
current validation loss is 0.976834801197052 andjaccard score is 61.79%
current validation loss is 0.976834801197052 andjaccard score is 61.31%
current validation loss is 0.976834801197052 andjaccard score is 61.70%
current validation loss is 0.9829003476179563 andjaccard score is 61.33%
current validation loss is 0.9829003476179563 andjaccar

In [23]:
# test
max_seq_length = 180

class Tweet_Loader_Test(object):
    '''
    1. tokenize the text
    2. find the indices of the selected text in the original text
    3. append the question to the original text and modify original indexes
    4. identify start index and end index
    5. identify start and end index
    '''

    def __init__(self, text, sentiment, textID):
        
        # define the text column from the dataframe
        self.text_column = text.tolist()
    
        # define sentiment
        self.sentinent = sentiment.tolist()
                   
        # define the label column and transform it to list
        self.textID = textID.tolist()
        
    # iter method to get each element at the time and tokenize it using bert        
    def __getitem__(self, index):
        
        sentinent = self.sentinent[index]
        text_id = self.textID[index]
        text = self.text_column[index]
        
        # get tokenized text
        tokenized_text = tokenizer.tokenize(text)
        
                
        # use the encode plus function
        dictionary_inputs = tokenizer.encode_plus(self.sentinent[index],
                                                  tokenized_text, max_length = max_seq_length,
                                                  truncation_strategy  ='do_not_truncate', 
                                                  pad_to_max_length = True, padding_side='Right', 
                                                  is_pretokenized = True, return_token_type_ids=True,
                                                  return_attention_mask=True,
                                                  return_tensors = 'pt')
        
        return [dictionary_inputs, text_id]
  
    def __len__(self):
        return len(self.text_column)

In [24]:
# create a class to process the traininga and test data
batch_size = 4
test_data = Tweet_Loader_Test(test['text'],test['sentiment'], test['textID'])

# use the dataloaders class to load the data
dataloaders_dict_test = {'test': DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)}

dataset_sizes = {'test':len(test_data)}

In [25]:
def predictions(model = None):
    prediction_data_frame_list = []
    
    with torch.no_grad():
        model.eval()
        
        for i, (dict_param, tweet_id) in enumerate(dataloaders_dict_test['test']):
            # add a counter that will register how many examples we have fed to the
            # model
            # move the sequences, labels and masks to the GPU
                inputs = dict_param.get('input_ids').to(device).squeeze()
                token_type_ids = dict_param.get('token_type_ids').to(device).squeeze()
                attention_mask = dict_param.get('attention_mask').to(device).squeeze()

            # feed the sequences to the model, specifying the attention mask
                start_scores, end_scores = model(inputs, token_type_ids = token_type_ids, 
                                                   attention_mask = attention_mask)
            
                start_preds=torch.argmax(start_scores,dim=1).cpu().detach().numpy()
                end_preds=torch.argmax(end_scores,dim=1).cpu().detach().numpy()
                inputs = inputs.cpu().detach().numpy()
                
                for i in range(inputs.shape[0]):
                    a = tweet_id[i]
                    pred_string = reconstruct_string(start_preds[i], end_preds[i], inputs[i])
                    pred_string = pred_string.replace(' ##', '')
                    print(f'the id is {a} and the string is {pred_string}')
                                        
                    temp_data = pd.DataFrame(zip([a],[pred_string]), columns = ['textID', 'selected_text'])
                    print(temp_data.head())
                    prediction_data_frame_list.append(temp_data)                
    
    prediction_df = pd.concat(prediction_data_frame_list)
    return prediction_df

In [26]:
model.load_state_dict(torch.load('../models_trained/bert_tweet_extraction_no_finetuned.pth'))


<All keys matched successfully>

In [27]:
prediction = predictions(model)

the id is f87dea47db and the string is last session of the day http : / / twitpic . com / 67ezh
       textID                                      selected_text
0  f87dea47db  last session of the day http : / / twitpic . c...
the id is 96d74cb729 and the string is exciting ( precisely - - skyscrapers galore ) . good tweeps in china : ( sh ) ( bj ) .
       textID                                      selected_text
0  96d74cb729  exciting ( precisely - - skyscrapers galore ) ...
the id is eee518ae67 and the string is recession hit veronique branquinho , she has to quit her company , such a shame !
       textID                                      selected_text
0  eee518ae67  recession hit veronique branquinho , she has t...
the id is 01082688c6 and the string is happy bday !
       textID selected_text
0  01082688c6  happy bday !
the id is 33987a8ee5 and the string is i like it ! !
       textID  selected_text
0  33987a8ee5  i like it ! !
the id is 726e501993 and the string is that ` s 

the id is e591a725ef and the string is i still dont know what we would do though
       textID                              selected_text
0  e591a725ef  i still dont know what we would do though
the id is 2dbb28cc29 and the string is dang it , so its not certain ? ? ? are you okay ?
       textID                                      selected_text
0  2dbb28cc29  dang it , so its not certain ? ? ? are you okay ?
the id is a7e4ad250a and the string is change of plans . i am staying in brandon . no papaya salad for me .
       textID                                      selected_text
0  a7e4ad250a  change of plans . i am staying in brandon . no...
the id is 7d87899024 and the string is dear rain , you suck .
       textID           selected_text
0  7d87899024  dear rain , you suck .
the id is 461b51fa58 and the string is at starbucks with my love . eff school . i have work later too .
       textID                                      selected_text
0  461b51fa58  at starbucks with my love 

the id is f9382a1f7b and the string is oooh , sunshine ! a patch of sunshine ! and it will be gone by the time i leave work and replaced with rain . / vent
       textID                                      selected_text
0  f9382a1f7b  oooh , sunshine ! a patch of sunshine ! and it...
the id is 54670ca807 and the string is skype call with billie but my webcam dont work
       textID                                   selected_text
0  54670ca807  skype call with billie but my webcam dont work
the id is 3ab338c459 and the string is just had a great study time followed by a delicious japanese meal with arty ! now , trying to get back into the study mood
       textID                                      selected_text
0  3ab338c459  just had a great study time followed by a deli...
the id is c7c7f7f025 and the string is fml not having a car is prohibiting finding a job
       textID                                      selected_text
0  c7c7f7f025  fml not having a car is prohibiting finding

the id is 6634c62acf and the string is hahaha ! alright . .
       textID         selected_text
0  6634c62acf  hahaha ! alright . .
the id is c7240790da and the string is good for you and congrats
       textID              selected_text
0  c7240790da  good for you and congrats
the id is 08ede23635 and the string is good to know thanks
       textID        selected_text
0  08ede23635  good to know thanks
the id is 4e87dcff0e and the string is what are you ? flavor of the day ?
       textID                       selected_text
0  4e87dcff0e  what are you ? flavor of the day ?
the id is 4b0b53bc98 and the string is ugh ! . . . hada gd nite tho !
       textID                   selected_text
0  4b0b53bc98  ugh ! . . . hada gd nite tho !
the id is fd3a622954 and the string is just listened to condre scr and got an invitation to watch their live concert on june 20th in munich http : / / bit . ly / daxa0 # postrock
       textID                                      selected_text
0  fd3a62295

the id is 2f883c6eb1 and the string is st joe is dirty .
       textID      selected_text
0  2f883c6eb1  st joe is dirty .
the id is a1320a5050 and the string is http : / / twitpic . com / 4w67k - camping at black butte lake
       textID                                      selected_text
0  a1320a5050  http : / / twitpic . com / 4w67k - camping at ...
the id is d72a60bee8 and the string is [SEP] are you getting my messages or do you have to add me to recieve them , soz i ` m new to this
       textID                                      selected_text
0  d72a60bee8  [SEP] are you getting my messages or do you ha...
the id is bc9ebe0002 and the string is i hate my life
       textID   selected_text
0  bc9ebe0002  i hate my life
the id is 64ddc1f8a2 and the string is sick to my stomach . . and i have a headache . . i wish someone could come rub my temples .
       textID                                      selected_text
0  64ddc1f8a2  sick to my stomach . . and i have a headache ....
th

...

the id is 08d12b46b2 and the string is its definitely coming , hope ur ready !
       textID                            selected_text
0  08d12b46b2  its definitely coming , hope ur ready !
the id is b9078fdb88 and the string is i have so much homework i think i am going to hide in a corner and cry .
       textID                                      selected_text
0  b9078fdb88  i have so much homework i think i am going to ...
the id is 6434edb2a5 and the string is sad that david is leaving tomoro . for a week .
       textID                                    selected_text
0  6434edb2a5  sad that david is leaving tomoro . for a week .
the id is 93c709ddbb and the string is a pleasure
       textID selected_text
0  93c709ddbb    a pleasure
the id is 261ead9c14 and the string is decaf isn ` t cutting it right now . ? http : / / blip . fm / ~ 79g85
       textID                                      selected_text
0  261ead9c14  decaf isn ` t cutting it right now . ? http : ...
the id is 4

the id is a29a6f5c41 and the string is happy mom ` s day .
       textID        selected_text
0  a29a6f5c41  happy mom ` s day .
the id is a4d9ee1aac and the string is in a really good mood for absolutely no reason tee her
       textID                                      selected_text
0  a4d9ee1aac  in a really good mood for absolutely no reason...
the id is 5729a6ebb9 and the string is hmm i guess where the world is headed 2day . . . . lets make predictions
       textID                                      selected_text
0  5729a6ebb9  hmm i guess where the world is headed 2day . ....
the id is b93d5198af and the string is so far i ` ve tried to kill myself twice in the 15 mins i ` ve been outside by tripping on stupid things . i knew i should ` ve stayed in bed .
       textID                                      selected_text
0  b93d5198af  so far i ` ve tried to kill myself twice in th...
the id is 771545d181 and the string is loong day . in bed cuddling with papabear and watchin

the id is 9ef3a57e41 and the string is i know ! i miss her . . .
       textID              selected_text
0  9ef3a57e41  i know ! i miss her . . .
the id is 2a20862961 and the string is it ` s been raining the past few days but it was actually sunny today ! well , for a little while . now it ` s super cloudy !
       textID                                      selected_text
0  2a20862961  it ` s been raining the past few days but it w...
the id is 218412b76a and the string is blessed to look forward to seeing family members visit .
       textID                                      selected_text
0  218412b76a  blessed to look forward to seeing family membe...
the id is 1d865c0717 and the string is i have the biggest headache ever . my photosensitivity is getting out of control ! ! help .
       textID                                      selected_text
0  1d865c0717  i have the biggest headache ever . my photosen...
the id is 7b474b659d and the string is what up ? why no tweets . are yo

the id is 77a7b3282b and the string is oh well i hope she gets better
       textID                   selected_text
0  77a7b3282b  oh well i hope she gets better
the id is 76b40b8c3d and the string is http : / / twitpic . com / 1aida - snood
       textID                             selected_text
0  76b40b8c3d  http : / / twitpic . com / 1aida - snood
the id is 1b6fe730fd and the string is maybe going to see the hannah montana movie todaaaay i ` ve seen it one time before , but can ` t wait to see it again ! !
       textID                                      selected_text
0  1b6fe730fd  maybe going to see the hannah montana movie to...
the id is 2a04e6e839 and the string is managed to find a bottle of pimm ` s ! if only we had the weather to go with it
       textID                                      selected_text
0  2a04e6e839  managed to find a bottle of pimm ` s ! if only...
the id is f944be482a and the string is congrats sarah ! ! ! ! ! thats awsum
       textID                

the id is 3a92bdf00a and the string is sooo gayy though
       textID     selected_text
0  3a92bdf00a  sooo gayy though
the id is d24ea15375 and the string is it was sunny when i entered the sub - basement . now it ` s raining .
       textID                                      selected_text
0  d24ea15375  it was sunny when i entered the sub - basement...
the id is c3c168bc80 and the string is orange vitamin water 10 is my new addiction u should try it .
       textID                                      selected_text
0  c3c168bc80  orange vitamin water 10 is my new addiction u ...
the id is 189ea0d26f and the string is yay thanku ! * hugs *
       textID          selected_text
0  189ea0d26f  yay thanku ! * hugs *
the id is fa117106ef and the string is so what was said ? ? i ` m so bummed
       textID                         selected_text
0  fa117106ef  so what was said ? ? i ` m so bummed
the id is f66bda14c6 and the string is i ¿ ½i ¿ ½we are ready for the new generation ! ! . . . 

In [28]:
# load model
prediction.to_csv('submission.csv', index=False)

In [30]:
#tokenizer.save_vocabulary('../models_trained/vocab.txt')

('../models_trained/vocab.txt',)